# ___Les Results___

---

## Le prototype de Result

- Un Result ressemble beaucoup à une Option à la différence qu'au lieu d'avoir un variant None qui désigne l'absence de quelque chose, un Result possède un variant Err pour erreur ainsi qu'un type d'erreur associé. Le variant Some(T) devient un variant Ok(T), ce qui exprime que le résultat est okay. Donc un Result a deux types génériques, celui de la valeur "correcte" qu'il contient soit `Ok(T)` et le type de l'erreur soit `Err(E)`.

![PROTO](pictures/proto_result.png)

> Comme pour les Option, les variants Ok et Err sont dans le prélude de Rust et n'ont donc pas besoin d'être préfixés par **Result::**.

## Creation et utilisation d'un Result

### Utilisation naive

- On serait tenté de faire un exemple simple pour Result mais l'on va se heurter à un problème de sens. Quel est le type de l'erreur ? Pourquoi construire une structure Result directement ?

In [12]:
let res: Result<usize, &str> = Ok(42); // Je dois preciser les deux types generiques...
dbg!(res);

let res: Result<usize, &str> = Err("Why am i coding this ?");
dbg!(res);

[src/lib.rs:122] res = Ok(
    42,
)
[src/lib.rs:125] res = Err(
    "Why am i coding this ?",
)


> Ce sont plutôt certaines fonctions de la std, des crates ou du programme principal qui retourneront des Result avec un type d'erreur détermine par le prototype de ces dernières. À part exception que je n'ai jamais croisé, l'on n'a pas a déclarer directement des variables de type Result avec leurs contenus...

### Fontion retournant un Result

- L'exemple ci-dessous ressemble davantage à un cas d'utilisation commun de Result.

In [19]:
fn check_if_lower_that_ten(v: usize) -> Result<(), &'static str> {
    if v < 10 {
        Ok(()) // Si tout est OK, on retourne un Tuple vide dans Ok ...
    } else {
        Err("The value is too big !") // ... sinon retourne une chaine de caractere dans Err
    }
}
dbg!(check_if_lower_that_ten(4));
dbg!(check_if_lower_that_ten(15));

[src/lib.rs:126] check_if_lower_that_ten(4) = Ok(
    (),
)
[src/lib.rs:127] check_if_lower_that_ten(15) = Err(
    "The value is too big !",
)


> Ne pas s'inquiéter du lifetime `static` utilise ici, les lifetimes seront abordes bien plus tard. Retenez pour l'instant qu'il est nécessaire à la compilation quand une fonction retourne une chaîne de caractère `&str` !

**Ici le type de retour et d'erreur de mon Result est déterminé par la fonction que j'appelle.**

### Avec une fonction de la std

- Essayons maintenant de parser des chaînes de caractère qui représentent des nombres entiers. D'après la documentation de Rust, la fonction parse() pour les chaînes de caractère ressemble a ceci :

![PARSE](pictures/parse_result.png)

**Explications:**
> - Ceci est d'abord la documentation du type primitif chaîne de caractère `str` donc self est une chaîne de caractère.
> - `F` **est un type generique**, c'est le type numérique vers lequel je veux parser. Il peut très bien être **u32**.
> - Seulement, je ne suis pas libre de prendre le type que je veux pour F, il y a une contrainte de trait exprimée par le `where`, en fait, le type de F DOIT IMPÉRATIVEMENT implémenter le trait `FromStr`, ce trait confère la capacité d'être parsee depuis une chaîne de caractère et il est en effet implémente pour u32.
> - Le second générique, le type d'erreur est **le type Err associe a FromStr**

**Bien que ce prototype soit plutôt complique à comprendre pour un débutant, il indique que le type d'erreur du Result sera FromStr::Err. Ainsi, lorsque l'on appelle la fonction parse, on me lui donne qu'un seul type, le type de l'erreur étant associe à ce dernier.**

#### **Utilisation d'un TURBOFISH pour donner le type :**

- Le turbofish `::<>` est une syntaxe agréable qui permet de donner le type au compilateur. Le **turbofish** se place toujours entre le nom de la fonction qui prend un type générique et les parenthèses et sa syntaxe ressemble à un poisson d'où le nom turbofish !

```
 o
o      ______/~/~/~/__           /((
  o  // __            ====__    /_((
    //  @))       ))))      ===/__((
    ))           )))))))        __((
    \\     \)     )))     __===\ _((
     \\_______________====      \_((
                                 \((

```

In [20]:
// On declare une chaine de caractere
// Elle ne contient que des chiffres
let a = "1637";
let res = a.parse::<u32>(); // On souhaite un u32.
dbg!(res);

// Ici, une chaine comppsee de chiffre et de lettre
let b = "16cents trente-sept";
let res = b.parse::<u32>();
dbg!(res);

[src/lib.rs:128] res = Ok(
    1637,
)
[src/lib.rs:133] res = Err(
    ParseIntError {
        kind: InvalidDigit,
    },
)


> Le type d'erreur étant un type associé, il n'est pas utile de le renseigner dans le turbofish. Ce sera toujours FromStr::Err.

#### **Declaration du type :**

- Les programmeurs Rust qui n'aiment pas utiliser le turbofish (et malheureusement, ils sont nombreux.) peuvent a la place donner le type de Result directement dans la déclaration de la variable de retour, le moteur d'inférence se chargera de remplacer T par ce type :

In [14]:
// On declare une chaine de caractere
// Elle ne contient que des chiffres
let a = "1637";
let res: Result<u32, _> = a.parse();
dbg!(res);

// Ici, une chaine comppsee de chiffre et de lettre
let b = "16cents trente-sept";
let res: Result<u32, _> = b.parse();
dbg!(res);

[src/lib.rs:124] res = Ok(
    1637,
)
[src/lib.rs:129] res = Err(
    ParseIntError {
        kind: InvalidDigit,
    },
)


> **L'underscore** dans la seconde partie du type demande au compilateur de se charger de l'inférer. Comme c'est un type associé, il n'y aura pas de problème.

> Cependant, il aurait pu être défini manuellement. C'est cependant complètement inutile !

In [21]:
let a = "1637";
let res: Result<u32, <u32 as std::str::FromStr>::Err> = a.parse();
dbg!(res);

[src/lib.rs:128] res = Ok(
    1637,
)


## La gestion des erreurs

**Que doit-on faire face à une erreur ? Il existe en Rust grosso modo deux façons de gérer les erreurs :**
> - **Paniquer**, le programme s'arrête brutalement, on utilise cela lorsque l'on écrit du code rapidement ou bien lorsque l'on considère que gérer l'erreur ne sert absolument à rien puisque le programme ne peut vraiment plus continuer après une telle erreur.
> - **Gerer l'erreur**, on gère proprement l'erreur en expliquant peut être bien ce qui s'est passé, l'on tente de la rattraper si possible ou on quitte calmement le programme.

### Le panic

- le programme, il cesse brutalement en laissant une trace qui peut nous enseigner ce qui s'est produit :

In [22]:
// Ici, une chaine comppsee de chiffre et de lettre
let b = "16cents trente-sept";
let number = b.parse::<u32>().unwrap();
dbg!(number);

thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: ParseIntError { kind: InvalidDigit }', src/lib.rs:127:31
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
   2: core::result::unwrap_failed
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/result.rs:1790:5
   3: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   4: run_user_code_20
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


- Il existe aussi une méthode nommée **expect()** qui a le même comportement, mais qui permet d'afficher tout de même un message custom sur la sortie d'erreur :
>```
> pub fn expect(self, msg: &str) -> T
> where
> E: Debug,
>```

- Expect permet de paniquer, mais en affichant une chaîne de caractère en plus de la backtrace :

In [24]:
// Ici, une chaine comppsee de chiffre et de lettre
let b = "16cents trente-sept";
let number = b.parse::<u32>().expect("Conversion is failed !");
dbg!(number);

thread '<unnamed>' panicked at 'Conversion is failed !: ParseIntError { kind: InvalidDigit }', src/lib.rs:125:31
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
   2: core::result::unwrap_failed
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/result.rs:1790:5
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Que veut dire ce **where E: Debug** dans le prototype d'expect et d'où vient-il ?

Déjà E fait référence au E de Result :
```
pub enum Result<T, E> {
    Ok(T),
    Err(E),
}
```

C'est une contrainte de trait pour le type d'erreur E, il doit impérativement implémenter Debug. Cette implémentation permet a expect d'afficher le variant Err soit **ParseIntError { kind: InvalidDigit }** ici.

### Traiter l'erreur

- On peut par exemple dans un match, exécuter une routine spécifique de code en cas d'erreur :

In [25]:
fn evaluate(s: &str) {
    let result = s.parse::<u32>();
    match result {
        Ok(value) => println!("Parsed Result is {}", value),
        Err(error) => println!("Sorry, but an error has occured: {}", error),
    }
}
evaluate("1637");
evaluate("16cents trente-sept");

Parsed Result is 1637
Sorry, but an error has occured: invalid digit found in string
